In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$Y \sim \mathcal{N}(np \, np\cdot(1 - p))\$$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для средней конверсии пользователей из контрольной выборки с уровнем значимости 95%. Округлите левую и правую границу с точностью до двух знаков после запятой. Запишите значения левой и правой границ через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой.

#### РЕШЕНИЕ

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

In [47]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

data = pd.read_csv('a_b_testing.csv')
print(data.columns)
print(data)
x = data[(data['group'] == 'A')]
print(x)
st_dev = x.groupby('group')['converted'].std()
sample_mean = x['converted'].mean()

def compute_ci(sample, st_dev):
    z_value = stats.norm.ppf(q = 0.95)
    sample_size = len(sample)
    
    #print("sample_size",sample_size)
    #print("sample_mean",sample_mean)
    #print("st_dev",st_dev)
    
    interval = z_value * (st_dev/np.sqrt(sample_size))
    conf_inv = (sample_mean - interval, sample_mean + interval)
    return conf_inv

print(compute_ci(x,st_dev))


Index(['converted', 'group'], dtype='object')
      converted group
0             1     A
1             0     A
2             0     A
3             1     B
4             1     A
...         ...   ...
3995          1     A
3996          0     A
3997          1     B
3998          0     B
3999          1     A

[4000 rows x 2 columns]
      converted group
0             1     A
1             0     A
2             0     A
4             1     A
7             0     A
...         ...   ...
3993          1     A
3994          0     A
3995          1     A
3996          0     A
3999          1     A

[2037 rows x 2 columns]
(group
A    0.393939
Name: converted, dtype: float64, group
A    0.429822
Name: converted, dtype: float64)


Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Задание по статистике Стьюдента 

Найдите значение статистики Стьюдента в предположении независимости выборок по формуле:

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

где `n` - размер контрольной выборки, `m`  - размер тестовой выборки.

Ответ запишите в переменную `answer2` с точностью до двух знаков после запятой.

### РЕШЕНИЕ

In [48]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

data = pd.read_csv('a_b_testing.csv')
print(data.columns)
print(data)
A = data[(data['group'] == 'A')]
B = data[(data['group'] == 'B')]
sA = A['converted'].std()
sB = B['converted'].std()
X = A['converted'].mean()
Y = B['converted'].mean()
def student(X,Y,sA,sB):
    return (X-Y)/np.sqrt(((sA*sA)/2037)+((sB*sB)/(4000-2037)))

print(student(X,Y,sA,sB))

Index(['converted', 'group'], dtype='object')
      converted group
0             1     A
1             0     A
2             0     A
3             1     B
4             1     A
...         ...   ...
3995          1     A
3996          0     A
3997          1     B
3998          0     B
3999          1     A

[4000 rows x 2 columns]
-1.6123875817242772


### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [49]:
from scipy.stats import ttest_ind
data = pd.read_csv('a_b_testing.csv')
print(data.columns)
print(data)
A = data[(data['group'] == 'A')]
B = data[(data['group'] == 'B')]
print(A['converted'])
print(B['converted'])
print(stats.ttest_ind(A['converted'].to_numpy(),
                      B['converted'].to_numpy()))

Index(['converted', 'group'], dtype='object')
      converted group
0             1     A
1             0     A
2             0     A
3             1     B
4             1     A
...         ...   ...
3995          1     A
3996          0     A
3997          1     B
3998          0     B
3999          1     A

[4000 rows x 2 columns]
0       1
1       0
2       0
4       1
7       0
       ..
3993    1
3994    0
3995    1
3996    0
3999    1
Name: converted, Length: 2037, dtype: int64
3       1
5       1
6       0
8       1
16      0
       ..
3976    1
3983    1
3985    0
3997    1
3998    0
Name: converted, Length: 1963, dtype: int64
Ttest_indResult(statistic=-1.6126205013707797, pvalue=0.10690590820177126)


Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать.

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой